In [6]:
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sdv.evaluation.single_table import run_diagnostic
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import get_column_plot
from sdv.evaluation.single_table import get_column_pair_plot
from sdv.evaluation.single_table import evaluate_quality
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import itertools
import yaml
from yaml.loader import SafeLoader
import mlflow
from sdv.single_table import CTGANSynthesizer
import mlflow.pyfunc
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

os.chdir('/home/onyxia/work/synthetic-data-sdc/')
os.environ["MLFLOW_TRACKING_URI"] = "https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/" 

In [2]:
variables = ['sex','age','agegr','placesize','edu','socprof','marital','ls','depress','trust','trustfam','trustneigh','sport','nofriend','smoke','alcabuse','alcsol','wkabint','englang','height','weight','bmi']
original = pd.read_csv('SDV/df_original.csv', names = variables)
sample = pd.read_csv('SDV/df1_sample.csv', names = variables)
cart = pd.read_csv('SDV/df1_cart.csv', names = variables)
ctree = pd.read_csv('SDV/df1_ctree.csv', names = variables)
parametric = pd.read_csv('SDV/df1_parametric.csv', names = variables)
rf = pd.read_csv('SDV/df1_rf.csv', names = variables)
bag = pd.read_csv('SDV/df1_bag.csv', names = variables)

In [3]:
num = ['age', 'depress', 'nofriend', 'height', 'weight', 'bmi']
fac = ['sex', 'agegr', 'placesize', 'edu', 'socprof', 'marital',
       'ls', 'trust', 'trustfam', 'trustneigh', 'sport', 'smoke',
       'alcabuse', 'alcsol', 'wkabint', 'englang']

In [7]:
ctgan_test = mlflow.pyfunc.load_model("runs:/d682b1f373e14cedbcac2ddff64ab9d2/ctgan_model")

original['t'] = 0
synthetique = ctgan_test.predict(original)
synthetique['t'] = 1

df_comb = pd.concat([original, synthetique], ignore_index=True)
df_comb_enc = pd.get_dummies(df_comb, columns=fac)
X = df_comb_enc.drop(columns=['t'])
y = df_comb_enc['t']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)
prop = model.predict(X)
pMSE = mean_squared_error(y, prop)
print(f'pMSE: {pMSE}')

pMSE: 0.00934173365196611


In [8]:
original['t'] = 0
cart['t'] = 1

df_comb1 = pd.concat([original, cart], ignore_index=True)
df_comb_enc1 = pd.get_dummies(df_comb1, columns=fac)
X1 = df_comb_enc1.drop(columns=['t'])
y1 = df_comb_enc1['t']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=1)

model1 = DecisionTreeClassifier(random_state=1)
model1.fit(X_train1, y_train1)
prop1 = model1.predict_proba(X1)[:, 1]
pMSE = mean_squared_error(y1, prop1)
print(f'pMSE: {pMSE}')

pMSE: 0.1596784705626765
